In [1]:
# power = 260mW 
# T = 64 C 
# %matplotlib ipympl 
%matplotlib qt 
%load_ext autoreload 
%autoreload 2
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.dmd import DMD
import numpy as np 
from pianoq.misc.mplt import mimshow, mplot
import datetime 
import time 
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.lee_holo.generate_lee import main, make_oam_donut, orthogonal_lee

In [ ]:
all_s1 = []
all_s2 = []
all_c = []
save_to = r"G:\My Drive\Projects\DMD\Alon results\Quantum\detectors_bug2.npz"
for i in range(100000):
    s1, s2, c = tt.read_interesting()
    s1, s2, c = s1.item(), s2.item(), c.item()
    all_s1.append(s1)
    all_s2.append(s2)
    all_c.append(c)
    print(f'{s1=}, {s2=}, {c=}')
    if i % 60 == 0:
        s1np = np.array(all_s1)
        s2np = np.array(all_s2)
        cnp = np.array(all_c)
        np.savez(save_to, s1=s1np, s2=s2np, c=cnp)


In [37]:
data = np.load(r"G:\My Drive\Projects\DMD\Alon results\Quantum\detectors_bug.npz")
# data['s1']
import matplotlib.pyplot as plt 
fig, ax = plt.subplots()
ax.plot(data['s1'], label='s1')
ax.plot(data['s2'], label='s2')
ax.plot(data['c'], label='c')
ax.legend()
fig.show()


In [19]:
all_c

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [2]:
# Laser@260mW 
dmd = DMD()
dmd.set_white()

Connected!
Built project!


In [ ]:
grtn = 70
dmd.set_grating(grtn, in_y=False)

# qq = 90
# dmd.set_checkerboard(qq)

# target = make_oam_donut(Nx=dmd.Nx, Ny=dmd.Ny, ell=2, r0=0.35, sigma=0.2)
# hologram, _ = orthogonal_lee(target, carrier_f=(1/50, 0), duty_cycle=0.5)
# dmd.set_image(hologram.astype(bool))

# amps = np.array([0, 0.5, 1])
# dmd.set_1d_macrosteps(amps, step_w=150, tile=8)

# dmd.set_black_between(450, 480)

# dmd.set_white()

mimshow(dmd.cur_image)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [ ]:
# mimshow(dmd.cur_image)
print(dmd.last_called_setter)

In [ ]:
mimshow(dmd.cur_image)

In [ ]:
timeout = 100000
x_motor = ThorlabsKcubeDC(serial_number=27253522, timeout=timeout)
y_motor = ThorlabsKcubeStepper(serial_number=26001271, timeout=timeout)

In [ ]:
print(x_motor.get_position())
print(y_motor.get_position())

In [9]:
integration_time = 1
# coin_window = 4e-9 
# ph = PhotonCounter(serial_port='COM8', integration_time=integration_time)
coin_window = 1.2e-9 # s 
# When DMD-plane fiber is 100um 1m. Delays=(0, 500) when both are 50um 2m fibers
tt = ph = QPTimeTagger(integration_time=integration_time, coin_window=coin_window, single_channel_delays=(0, 5000), remote=True, address='132.64.81.123:41101')  

In [36]:
tt.show_gui()

In [ ]:
tt.set_integration_time(1)
tt.show_gui()

In [11]:
s1, s2, c = ph.read_interesting()
s1, s2, c = s1.item(), s2.item(), c.item()
acc = 2*s1*s2*coin_window
real_c = c - acc 
std = np.sqrt(c) / np.sqrt(ph.integration_time)
print(f'{c=:.1f}; {real_c=:.1f}+-{std:.1f}')
print(x_motor.get_position())
print(y_motor.get_position())

c=0.0; real_c=0.0+-0.0


NameError: name 'x_motor' is not defined

In [ ]:
x_motor.move_absolute(13.34)

In [ ]:
# time.sleep(20)
start_x = 11.5 
start_y = 10.2 
x_pixels = 30
y_pixels = 3
pixel_size_x = 0.1  # mm
pixel_size_y = 0.1  # mm
name = f'grating_100_timetagger_z_12_fiber_100um_fiber_50um_260mW_f_none_before_crystal_filter_20nm'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\Projects\DMD\Alon results\Quantum\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=True, coin_window=coin_window, saveto_path=path)

single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=tt)

In [ ]:
# time.sleep(10)
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=tt)

In [ ]:
mplot(scanner.result.real_coins.sum(axis=0))

In [ ]:
# x_motor.close()
# y_motor.close()
# ph.close()
# dmd.close()

In [ ]:
# dmd.set_grating(100)
dmd.set_white()

## ASI cam 

In [3]:
cam = ASICam()

In [12]:
path = r'G:\My Drive\Projects\DMD\Alon results\Quantum'
# cam.set_roi(4300, 2100, 500, 500)
cam.set_exposure(3)
cam.set_roi(None, None, None, None)
cam.set_binning(4)

dmd.set_grating(100)
cam.show_image(vmin=None, vmax=None)
cam.save_image(rf'{path}\farfield_plane_yes_lens_75mm_f1_200mm_3nm_grating50.fits')


# for i in [10, 30, 50, 70, 90, 110, 130, 150, 170, 190]:
#     dmd.set_grating(i)
#     cam.save_image(rf'{path}\farfield_plane_yes_lens_75mm_f1_200mm_3nm_grating{i}.fits')

In [ ]:
cam.close()

In [ ]:
# pump with f=75mm 
mask = np.ones(dmd.shape, dtype=np.uint8)
mask[:200, :] = 0 
mask[700:, :] = 0 
mask[:, :350] = 0 
mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
mask = np.ones(dmd.shape, dtype=np.uint8)
# mask[:200, :] = 0 
# mask[700:, :] = 0 
# mask[:, :350] = 0 
# mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
r = 450
X0 = 0
Y0 = 0
mask = np.zeros(dmd.shape, dtype=np.uint8)
mask[np.sqrt((dmd.XX-X0)**2+(dmd.YY-Y0)**2) < r] = 1 
dmd.set_image(mask)
# mimshow(mask)